In [2]:
import pandas as pd
import numpy as np 
import re
import random
import copy

In [3]:
# The file that includes the slot and intent information.
df = pd.read_excel("Appliance Slot Intent.xlsx",encoding = "ISO-8859-1", sheet_name="Main V5.")
# The file that includes the types of values inside that slot. 
slot_df = pd.read_excel("Appliance Slot Intent.xlsx",encoding = "ISO-8859-1", sheet_name="Slot Description V4.")

In [4]:
# The file that includes the slot and intent information.
df = pd.read_excel("Appliance Slot-Intent Manual.xlsx",encoding = "ISO-8859-1", sheet_name="Sheet1")
# The file that includes the types of values inside that slot. 
slot_df = pd.read_excel("Appliance Slot-Intent Manual.xlsx",encoding = "ISO-8859-1", sheet_name="Sheet3")

In [5]:
df

,QUESTION_LABEL,TARGET_QUESTION,INTENT_DESCRIPTION,EXAMPLE_SENTENCE,QUESTION_SHOULD_CONTAIN,Unnamed: 5
0,buy,Where can I buy a @partType for my @appliance?...,The sentence specifies that the user wants to ...,Where can I buy a glass tray for my microwave?...,@partType @appliance,NaN
1,clean,How do I clean the @cleanObject on the @applia...,The sentence asks how to clean a part of the a...,How do I clean the glass cooktop on the oven?\...,@cleanObject @appliance,NaN
2,code,What does @infoCode mean on my @appliance?\nMy...,The sentence is asking about the\nmeaning of t...,What is the c-d0 mean on my oven?\nMy microwav...,@infoCode @appliance,Information Code
3,appropriate,Is it safe to use @cookWare in a @appliance?\n...,A sentence that asks if it is safe to use a ce...,Is it safe to use Glassware in the microwave?\...,@cookWare @appliance,NaN
4,remove,How do I remove @removeObject from the @applia...,The sentence asks how to remove something\nfro...,How do I remove burned-on residue from the ove...,@removeObject @appliance,NaN
5,replace,How do I reinstall @replaceObject from the @ap...,The sentence asks how to reinstall something\n...,How do I reinstall the storage drawer from the...,@replaceObject @appliance,NaN
6,cook,What must I do to cook @foodType with the @app...,The sentence asks how to prepare a certain typ...,What must I do to cook popcorn with the microw...,@foodType @appliance,NaN
7,purpose,What is the purpose of the @purposeFeature on ...,The sentence asks for a purpose of a feature.\...,What is the purpose of the warming center on t...,@purposeFeature @appliance,NaN
8,set,How do I set the @setFeature feature on my @ap...,The sentence specifies that the user wants to ...,How do I set the Timer on the microwave?,@setFeature @appliance,NaN
9,use,How can I use the @useFeature on the @applianc...,A sentence that describes the user wanting\nto...,How can I use the soften feature on the microw...,@useFeature @appliance,NaN


In [6]:
slot_df.head()

,Slot,Description,Examples,Extra Explanation
0,@appliance,Any type of appliance. This slot is \nused to ...,"microwavawe, oven",This slot describes the appliance that the use...
1,@partType,A Physical part from an appliance.\nSomething ...,"door, light, wirerack, air fry tray, storage d...",The following slot values are items that can b...
2,@cleanObject,Parts that have specific cleaning instructions,"grease filter, cooktop, cooktop seal, door gas...",NaN
3,@infoCode,An error code or information code\ndisplayed i...,"c-d0, c-d1, c-F0, c-20, c-21, c-22, c-30, c-31...",NaN
4,@cookWare,A device to cook food,"aluminum foil, browning plate, ceramic, porcel...",NaN


In [7]:
df['QUESTION_SHOULD_CONTAIN']= df['QUESTION_SHOULD_CONTAIN'].str.lower()
slot_df['Slot']= slot_df['Slot'].str.lower()

In [8]:
df['TARGET_QUESTION']= df['TARGET_QUESTION'].str.lower()

In [9]:
df["TARGET_QUESTION"] = df['TARGET_QUESTION'].str.replace('[\.\?]','')

In [10]:
slot_df['Slot']= slot_df['Slot'].str.lower()

In [11]:
df.head()

,QUESTION_LABEL,TARGET_QUESTION,INTENT_DESCRIPTION,EXAMPLE_SENTENCE,QUESTION_SHOULD_CONTAIN,Unnamed: 5
0,buy,where can i buy a @parttype for my @appliance\...,The sentence specifies that the user wants to ...,Where can I buy a glass tray for my microwave?...,@parttype @appliance,NaN
1,clean,how do i clean the @cleanobject on the @applia...,The sentence asks how to clean a part of the a...,How do I clean the glass cooktop on the oven?\...,@cleanobject @appliance,NaN
2,code,what does @infocode mean on my @appliance\nmy ...,The sentence is asking about the\nmeaning of t...,What is the c-d0 mean on my oven?\nMy microwav...,@infocode @appliance,Information Code
3,appropriate,is it safe to use @cookware in a @appliance\ni...,A sentence that asks if it is safe to use a ce...,Is it safe to use Glassware in the microwave?\...,@cookware @appliance,NaN
4,remove,how do i remove @removeobject from the @applia...,The sentence asks how to remove something\nfro...,How do I remove burned-on residue from the ove...,@removeobject @appliance,NaN


In [12]:
## Preprocessing slot_df dataframe
# Slot dataframe trimmed so that the Description is removed. 
slot_df_clean = slot_df.iloc[:,[0,2]]
# replace the Example column with an array of the values separated by a comma
slot_df_clean = slot_df_clean[slot_df_clean['Examples'].notnull()].copy()
slot_df_clean.loc[:,'Examples'] = slot_df_clean.loc[:,"Examples"].apply(lambda x: x.split(', '))
# dictionary of the slots. Key: slot value: possible vallues of the slot. 
slot_dict = dict(zip(slot_df_clean.Slot, slot_df_clean.Examples))

In [13]:
slot_dict

{'@appliance': ['microwavawe', 'oven'],
 '@parttype': ['door',
  'light',
  'wirerack',
  'air fry tray',
  'storage drawer',
  'graphite lubricant',
  'glass tray',
  'roller guide ring',
  'exhaust filter',
  'grease filter',
  'charcoal filter',
  'hardware kit',
  'screws',
  'brackets'],
 '@cleanobject': ['grease filter',
  'cooktop',
  'cooktop seal',
  'door gasket',
  'stainless steel',
  'glass cooktop',
  'sugary-spills',
  'melted-plastic'],
 '@infocode': ['c-d0',
  'c-d1',
  'c-F0',
  'c-20',
  'c-21',
  'c-22',
  'c-30',
  'c-31',
  'c-A2',
  'bAd-LInE'],
 '@cookware': ['aluminum foil',
  'browning plate',
  'ceramic',
  'porcelain',
  'stoneware',
  'polyester cardboard',
  'polyester cup',
  'paper bags',
  'recycled paper',
  'metal trimes',
  'glassware',
  'metal',
  'paper',
  'plastic',
  'thermometer',
  'straw',
  'wicker',
  'wood'],
 '@removeobject': ['rack', 'residue', 'stains', 'door', 'drawer', 'lights'],
 '@replaceobject': ['rack',
  'door',
  'drawer',
  'l

In [14]:
slot_df_clean

,Slot,Examples
0,@appliance,"[microwavawe, oven]"
1,@parttype,"[door, light, wirerack, air fry tray, storage ..."
2,@cleanobject,"[grease filter, cooktop, cooktop seal, door ga..."
3,@infocode,"[c-d0, c-d1, c-F0, c-20, c-21, c-22, c-30, c-3..."
4,@cookware,"[aluminum foil, browning plate, ceramic, porce..."
5,@removeobject,"[rack, residue, stains, door, drawer, lights]"
6,@replaceobject,"[rack, door, drawer, lights, cooktop, night li..."
7,@foodtype,"[hamburger, beef steak, chicken pieces, pork c..."
8,@purposefeature,"[hot surface indicator, cooktop lamp, display,..."
9,@setfeature,"[clock, kitchen timer, timed cooking, delay st..."


In [15]:
# Split the TARGET_QUESTION column and make it into an array of questions
df.iloc[:,1] = df.iloc[:,1].apply(lambda x: x.split('\n'))
#Temporary extract the TARGET_QUESTION column that was processed above. 
target = df.iloc[:,1]

In [16]:
target

0     [where can i buy a @parttype for my @appliance...
1     [how do i clean the @cleanobject on the @appli...
2     [what does @infocode mean on my @appliance, my...
3     [is it safe to use @cookware in a @appliance, ...
4     [how do i remove @removeobject from the @appli...
5     [how do i reinstall @replaceobject from the @a...
6     [what must i do to cook @foodtype with the @ap...
7     [what is the purpose of the @purposefeature on...
8     [how do i set the @setfeature feature on my @a...
9     [how can i use the @usefeature on the @applian...
10    [how do i switch the @onofffeature off on the ...
Name: TARGET_QUESTION, dtype: object

In [17]:
target[0]

['where can i buy a @parttype for my @appliance',
 'i want to get a new @parttype for my @appliance',
 'how can i purchase a @parttype for my @appliance',
 'where can i find a new @parttype for my @appliance']

In [18]:
# A Series of arrays representing the respective slots
slots = df.iloc[:,4].apply(lambda x: x.split(' '))
# The dictionary where key:Intent value:Question
target_dic = df.iloc[:,[0,1]]
# A dictionary of intents key: int value: intent
intent_dict = df.QUESTION_LABEL.to_dict()

In [19]:
slots

0           [@parttype, @appliance]
1        [@cleanobject, @appliance]
2           [@infocode, @appliance]
3           [@cookware, @appliance]
4       [@removeobject, @appliance]
5      [@replaceobject, @appliance]
6           [@foodtype, @appliance]
7     [@purposefeature, @appliance]
8         [@setfeature, @appliance]
9         [@usefeature, @appliance]
10      [@onofffeature, @appliance]
Name: QUESTION_SHOULD_CONTAIN, dtype: object

In [20]:
slot_dict_boi = copy.deepcopy(slot_dict)
for key,value in slot_dict.items():
    list_slot = []
    for i in value:
        s = i.split()
        for j in range(len(s)):
            if j == 0:
                s[j] = s[j] + ":B-" + key[1:]
            else:
                s[j] = s[j] + ":I-" + key[1:]
        i = ' '.join(s)
        list_slot.append(i)
    slot_dict_boi[key] = list_slot

In [21]:
slot_dict_boi

{'@appliance': ['microwavawe:B-appliance', 'oven:B-appliance'],
 '@parttype': ['door:B-parttype',
  'light:B-parttype',
  'wirerack:B-parttype',
  'air:B-parttype fry:I-parttype tray:I-parttype',
  'storage:B-parttype drawer:I-parttype',
  'graphite:B-parttype lubricant:I-parttype',
  'glass:B-parttype tray:I-parttype',
  'roller:B-parttype guide:I-parttype ring:I-parttype',
  'exhaust:B-parttype filter:I-parttype',
  'grease:B-parttype filter:I-parttype',
  'charcoal:B-parttype filter:I-parttype',
  'hardware:B-parttype kit:I-parttype',
  'screws:B-parttype',
  'brackets:B-parttype'],
 '@cleanobject': ['grease:B-cleanobject filter:I-cleanobject',
  'cooktop:B-cleanobject',
  'cooktop:B-cleanobject seal:I-cleanobject',
  'door:B-cleanobject gasket:I-cleanobject',
  'stainless:B-cleanobject steel:I-cleanobject',
  'glass:B-cleanobject cooktop:I-cleanobject',
  'sugary-spills:B-cleanobject',
  'melted-plastic:B-cleanobject'],
 '@infocode': ['c-d0:B-infocode',
  'c-d1:B-infocode',
  'c-F0

In [22]:
def boi_save(intent_number = len(target), single = False):
    f = open("appliance2\\train_test", "w")
    f_train = open('appliance2\\train','w')
    f_test = open('appliance2\\test','w')
    f_valid = open('appliance2\\valid','w')
    with open('appliance2\\train','w') as f_train, open('appliance2\\test','w') as f_test, open('appliance2\\valid','w') as f_valid:
        line = ""
        start = 0
        end = intent_number
        if (single == True):
            start = intent_number - 1
            end = intent_number
        for s in range(start,end): #Intent number
    #         print("-----------------------------------------------")
            print(target[s])
            print("Intent:",intent_dict[s],"|","number of slots:",len(slots[s]))
            for j in range(len(target[s])): #Sentence in the intent
                if (len(slots[s]) == 2):
                    line,_ = boi_two_slots_write(s,j,f_train,f_test,f_valid)    
                elif (len(slots[s]) == 3):
                    line,_ = boi_three_slots_write(s,j,f_train,f_test,f_valid)
                elif (len(slots[s]) == 1):
                    line,_ = boi_one_slot_write(s,j,f_train,f_test,f_valid)
    #             print("-----------------------------------------------")
    #     print("-----------------------------------------------")
        f_train.close()
        f_test.close()
        f_valid.close()

def boi_one_slot_write(s,j,f):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
            x = target[s][j]
            x = re.sub(slots[s][0], slot_dict_boi[slots[s][0]][i], x)
            x_boi = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
            p += 1
            #print(x_boi)
            if((j % 4 == 0) | (j % 1 == 1)):
                f.write(x_boi + " <=> " + intent_dict[s] + "\n")
            elif (j % 4 == 3):
                f.write(x_boi + " <=> " + intent_dict[s] + "\n")
            else:
                f.write(x_boi + " <=> " + intent_dict[s] + "\n")
    return x, p

def boi_two_slots_write(s,j,f_train,f_test,f_valid):
    p = 0
    x_boi = ""
    for i in range(len(slot_dict_boi[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict_boi[slots[s][1]])):#Slot 2
            x_original = target[s][j]
            x = re.sub(slots[s][0], slot_dict_boi[slots[s][0]][i], x_original)
            x = re.sub(slots[s][1], slot_dict_boi[slots[s][1]][k], x)
            x_boi = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
            p += 1
            #print(x_boi)
            if((j % 4 == 0) or (j % 4 == 1)):
                f_train.write(x_boi + " <=> " + intent_dict[s] + "\n")
            elif (j % 4 == 2):
                f_test.write(x_boi + " <=> " + intent_dict[s] + "\n")
            else:
                f_valid.write(x_boi + " <=> " + intent_dict[s] + "\n")

    return x, p

def boi_three_slots_write(s,j,f):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict[slots[s][1]])):#Slot 2
            for l in range(len(slot_dict[slots[s][2]])):#Slot 3
                x_original = target[s][j]
                x = re.sub(slots[s][0], slot_dict_boi[slots[s][0]][i], x_original)
                x = re.sub(slots[s][1], slot_dict_boi[slots[s][1]][k], x)
                x = re.sub(slots[s][2], slot_dict_boi[slots[s][2]][l], x)
                x_boi = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
                p += 1
                f.write(x_boi+" <=> "+ intent_dict[s] + "\n")
                #print(x_boi)   
    return x, p

In [23]:
boi_save()

['where can i buy a @parttype for my @appliance', 'i want to get a new @parttype for my @appliance', 'how can i purchase a @parttype for my @appliance', 'where can i find a new @parttype for my @appliance']
Intent: buy | number of slots: 2
['how do i clean the @cleanobject on the @appliance', 'tell me how to polish the @cleanobject with the @appliance', 'what is the best way to clean the @cleanobject on the @appliance', 'i would like to clean the @cleanobject with the @appliance please']
Intent: clean | number of slots: 2
['what does @infocode mean on my @appliance', 'my @appliance has the @infocode message', 'what is the meaning of the @infocode message on my @appliance', 'how do i handle the @infocode message that is showing on my @appliance']
Intent: code | number of slots: 2
['is it safe to use @cookware in a @appliance', 'is the @cookware @appliance safe', 'is @cookware appropriate for the @appliance', 'can i use the @cookware for the @appliance']
Intent: appropriate | number of s

In [24]:
f_train = open('appliance2\\train','r')
f_test = open('appliance2\\test','r')
f_valid = open('appliance2\\valid','r')

# for i in f_train:
#     print(i)
    
print(len(f_train.readlines()))

f_train.close()
f_test.close()
f_valid.close()

688


In [25]:
f_test = open('appliance2\\test','r')
for i in f_test:
    print(i)
f_test.close()

how:O can:O i:O purchase:O a:O door:B-parttype for:O my:O microwavawe:B-appliance <=> buy

how:O can:O i:O purchase:O a:O door:B-parttype for:O my:O oven:B-appliance <=> buy

how:O can:O i:O purchase:O a:O light:B-parttype for:O my:O microwavawe:B-appliance <=> buy

how:O can:O i:O purchase:O a:O light:B-parttype for:O my:O oven:B-appliance <=> buy

how:O can:O i:O purchase:O a:O wirerack:B-parttype for:O my:O microwavawe:B-appliance <=> buy

how:O can:O i:O purchase:O a:O wirerack:B-parttype for:O my:O oven:B-appliance <=> buy

how:O can:O i:O purchase:O a:O air:B-parttype fry:I-parttype tray:I-parttype for:O my:O microwavawe:B-appliance <=> buy

how:O can:O i:O purchase:O a:O air:B-parttype fry:I-parttype tray:I-parttype for:O my:O oven:B-appliance <=> buy

how:O can:O i:O purchase:O a:O storage:B-parttype drawer:I-parttype for:O my:O microwavawe:B-appliance <=> buy

how:O can:O i:O purchase:O a:O storage:B-parttype drawer:I-parttype for:O my:O oven:B-appliance <=> buy

how:O can:O i

In [26]:
# Create appliance vocabulary for slots. 

slot_array = [] 
for key,value in slot_dict_boi.items():
    for i in value:
        slot_array = slot_array + i.split()
slot_array.sort()
slot_vocab = []
for i in slot_array:
    slot_vocab.append(i.split(':',1)[1])
    
slot_vocab.sort()
slot_vocab = list(dict.fromkeys(slot_vocab))

# for i in slot_vocab:
#     print(i)
    
f = open('appliance2\\vocab.slot','w')
for value in slot_vocab:
    f.write(value + "\n")
f.write("O")
f.close()

# Create appliance vocabulary for intents

f = open('appliance2\\vocab.intent','w')
for key,value in intent_dict.items():
    f.write(value + "\n")
f.close()

In [30]:
intent_dict.items()

dict_items([(0, 'buy'), (1, 'clean'), (2, 'code'), (3, 'appropriate'), (4, 'remove'), (5, 'replace'), (6, 'cook'), (7, 'purpose'), (8, 'set'), (9, 'use'), (10, 'on_off')])

In [28]:
type(content)

NameError: name 'content' is not defined

In [ ]:
print(len(train))
print(len(test))
print(len(valid))

In [ ]:
my_file = open('appliance2\\train_test','r')
content = my_file.readlines()
print(len(content))
train = content[:int(len(content)*0.7)]
test = content[int(len(content)*0.7):int(len(content)*0.9)]
valid = content[int(len(content)*0.9):]

train = content[::2]
test = content[1::4]
valid = content[3::4]
print(len(train))
print(len(test))
print(len(valid))


f = open('appliance2\\train','w')
for i in train:
    f.write(i)
    
f = open('appliance2\\test','w')
for i in test:
    f.write(i)
    
f = open('appliance2\\valid','w')
for i in test:
    f.write(i)

In [ ]:
train